In [1]:
import zap
import pypsa
from zap.tests import network_examples as examples

In [2]:
pn = examples.load_example_network('texas_7node')
pn

INFO:pypsa.io:Imported network texas_7node.nc has buses, carriers, generators, lines, loads, storage_units


PyPSA Network
Components:
 - Bus: 7
 - Carrier: 17
 - Generator: 176
 - Line: 12
 - Load: 7
 - StorageUnit: 14
Snapshots: 48

In [3]:
net, devices = zap.importers.load_pypsa_network(pn, power_unit=1e3, cost_unit=10.0)

INFO:zap.importers.pypsa:Importing Generator with 136 devices.
INFO:zap.importers.pypsa:Importing Load with 7 devices.
INFO:zap.importers.pypsa:Importing ACLine with 12 devices.
INFO:zap.importers.pypsa:Importing StorageUnit with 14 devices.


In [4]:
net.dispatch(devices=devices, time_horizon=48)

/Users/kamrantehranchi/Library/Caches/pypoetry/virtualenvs/zap-qb0btW3r-py3.12/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


DispatchOutcome(phase_duals=[None, None, [array([[ 3.62649101e-10,  6.42087774e-10,  6.33926573e-10,
         6.05645006e-10,  9.29180190e-10,  6.59538102e-10,
         8.23288164e-10,  8.32573953e-10,  6.78496589e-10,
         6.64165713e-10,  7.63642368e-10,  7.12555372e-10,
         8.99619855e-10,  3.95992699e-10,  4.73822600e-10,
         1.26063679e-10, -2.17658720e-10, -3.89927464e-10,
        -4.69235538e-10, -4.74652150e-10, -4.59811823e-10,
        -3.97925884e-10, -2.02157224e-10,  1.01656333e-10,
         3.48969538e-10,  5.53266312e-10,  5.17407367e-10,
         4.03562791e-10,  9.32890113e-10,  7.47605118e-10,
         7.87796033e-10,  8.38772162e-10,  6.86959106e-10,
         6.64040966e-10,  7.69403409e-10,  7.31365299e-10,
         8.45862968e-10,  3.85136429e-10,  4.57652921e-10,
         2.24871569e-10, -5.16614913e-11, -2.08212867e-10,
        -2.73666863e-10, -2.56684887e-10, -1.18862373e-10,
         7.72566376e-11,  1.76616413e-10,  3.47045850e-10],
       [-2.28